# Indice espectrales NDWI Analisis en GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio SHP

In [3]:
import os

In [4]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP\Indices"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\Indices'

In [5]:
import geopandas as gpd

In [6]:
# cargar un shp
zona = gpd.read_file("Zona_Agua_Geo.shp")
zona

,id,geometry
0,1,"POLYGON ((-52.07729 -2.97321, -51.51219 -2.967..."


In [7]:
# Crear un featureCollecion mediante GeoJSON
ee_zona = ee.FeatureCollection(zona.__geo_interface__)

In [12]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [14]:
geometria = ee_zona.geometry()
Map.centerObject(geometria, 9)

In [13]:
# Visualizar 
Map.addLayer(ee_zona, {"color" : "00FF11"}, name = "Zona")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Seleccion de imagenes

In [15]:
# Seleccion de la imagen Landsat 5 - 2011
img_L5 = ee.Image("LANDSAT/LT05/C01/T1_SR/LT05_225062_20110704")\
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
                    ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
            .multiply(0.0001)\
            .reproject(crs="EPSG:32722", scale = 30)\
            .clip(ee_zona)

In [16]:
# Seleccion de la imagen Landsat 8 - 2017
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_225062_20170720")\
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
                    ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
            .multiply(0.0001)\
            .reproject(crs="EPSG:32722", scale = 30)\
            .clip(ee_zona)

In [17]:
# Nombre de las bandas
print("Nombre bandas L5: ",img_L5.bandNames().getInfo())
print("Nombre bandas L8: ",img_L8.bandNames().getInfo())

Nombre bandas L5:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Nombre bandas L8:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [18]:
# Consultar Proyeccion
print("Proyeccion L5: ",img_L5.projection().getInfo())
print("Proyeccion L8: ",img_L8.projection().getInfo())

Proyeccion L5:  {'type': 'Projection', 'crs': 'EPSG:32722', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion L8:  {'type': 'Projection', 'crs': 'EPSG:32722', 'transform': [30, 0, 0, 0, -30, 0]}


In [19]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [20]:
# Visualizar 
Map.addLayer(img_L5, viz_es, name = "Landsat5 RS")
Map.addLayer(img_L8, viz_es, name = "Landsat8 RS")
Map

Map(bottom=133765.72729492188, center=[-3.2854821745049207, -51.519855776033324], controls=(WidgetControl(opti…

## Determinar NDWI

In [21]:
# Funcion para NDWI MF
def Calc_NDWI_mf(image):
    ndwi = image.expression("float(green - nir)/(green + nir)", {
        'green': image.select('GREEN'),
        'nir': image.select('NIR')
    })
    return ndwi.rename("NDWI")

In [22]:
# Calcular indices con funcion
ndwi_L5 = Calc_NDWI_mf(img_L5)
ndwi_L8 = Calc_NDWI_mf(img_L8)

In [23]:
# Simbologia de NDWI
viz_ndwi = {
    'palette': ['fff947', 'b9fa3e', '5ce5d3', '5172f4', '001334'],
    'min': -0.5,
    'max': 0.8,
    'bands': "NDWI"
}

In [24]:
# Visualizar 
Map.addLayer(ndwi_L5, viz_ndwi, name = "L5 NDWI")
Map.addLayer(ndwi_L8, viz_ndwi, name = "L8 NDWI")
Map

Map(bottom=133766.0, center=[-3.2854821745049207, -51.519855776033324], controls=(WidgetControl(options=['posi…

## Seleccionar Agua

In [25]:
#  NDWI >= 0.1 es agua
ndwi_L5_Agua = ndwi_L5.gte(0.1)
ndwi_L8_Agua = ndwi_L8.gte(0.1)

In [26]:
# Enmascarar NDWI con el agua
ndwi_L5_mascara = ndwi_L5.updateMask(ndwi_L5_Agua)
ndwi_L8_mascara = ndwi_L8.updateMask(ndwi_L8_Agua)

In [27]:
# Visualizar 
Map.addLayer(ndwi_L5_mascara, viz_ndwi, name = "L5 NDWI Mask")
Map.addLayer(ndwi_L8_mascara, viz_ndwi, name = "L8 NDWI Mask")
Map

Map(bottom=133766.0, center=[-3.2854821745049207, -51.519855776033324], controls=(WidgetControl(options=['posi…

## Exportar Imagen

In [28]:
import os
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\Indices'

In [29]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [30]:
# Exportar imagen directorio
#geemap.ee_export_image(ndwi_L5_mascara, filename="BRASIL_ndwi_L5_mascara.tif", scale=30, region=geometria, file_per_band=False)
#geemap.ee_export_image(ndwi_L8_mascara, filename="BRASIL_ndwi_L8_mascara.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
An error occurred while downloading.
Total request size (42558832 bytes) must be less than or equal to 33554432 bytes.
Generating URL ...
An error occurred while downloading.
Total request size (42558832 bytes) must be less than or equal to 33554432 bytes.


In [31]:
# Exportar en google drive
geemap.ee_export_image_to_drive(ndwi_L5_mascara, description='BRASIL_ndwi_L5_mascara', folder='GEE_Python', region=geometria, scale=30)
geemap.ee_export_image_to_drive(ndwi_L8_mascara, description='BRASIL_ndwi_L8_mascara', folder='GEE_Python', region=geometria, scale=30)

Exporting BRASIL_ndwi_L5_mascara ...
Exporting BRASIL_ndwi_L8_mascara ...
